In [1]:
import os
import json
import pandas as pd
import pysrt
import re

In [2]:
def compute_iou(interval_1, interval_2):
    start_i, end_i = float(interval_1[0]), float(interval_1[1])
    start, end = float(interval_2[0]), float(interval_2[1])
    intersection = max(0, min(end, end_i) - max(start, start_i))
    union = min(max(end, end_i) - min(start, start_i), end-start + end_i-start_i)
    iou = float(intersection) / (union + 1e-8)
    return iou

In [3]:
def save_json(content, save_path):
    with open(save_path, 'w') as f:
        f.write(json.dumps(content))
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(l.strip("\n")) for l in f.readlines()]
def load_result_json(filename):
    with open(filename, "r") as f:
        data = json.load(f)
    #print(data)
    return data

BASELINE Evaluation

In [4]:
val = load_jsonl(f'/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/all_data_val.json')
test = load_jsonl(f'/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/all_data_test.json')

In [5]:
folders = ['/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_gpt_query_type_experiment/baseline', 
           '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_gpt_query_type_experiment/visual_med', 
           #'/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_gpt_query_type_experiment/visual_nonmed', 
           '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_gpt_query_type_experiment/visual_med_with_audio']

In [6]:
test_med_queries_path = "/home/hlpark/shared/TVQA/output_gpt_tvqa_test_queries.txt"
val_med_queries_path = "/home/hlpark/shared/TVQA/output_gpt_tvqa_val_queries.txt"
test_med_query_f_metamap_list, val_med_query_f_metamap_list = [], []
with open(test_med_queries_path, "r") as f:
    lines = f.read()
    for line in lines.split("\n"):
        if line == "\n" or line == "":
            continue
        if line.startswith(" ") or line.startswith("-"):
            line = line.lstrip("-").lstrip(" ")
        test_med_query_f_metamap_list.append(line.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(",", "").replace(".", "").replace("/", "").replace(">", ""))
f.close()
with open(val_med_queries_path, "r") as f:
    lines = f.read()
    for line in lines.split("\n"):
        if line == "\n" or line == "":
            continue
        # if line.startswith(" ") or line.startswith("-"):
        #     line = line.lstrip("-").lstrip(" ")
        val_med_query_f_metamap_list.append(line.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(",", "").replace(".", "").replace("/", "").replace(">", ""))
        #val_med_query_f_metamap_list.append(line.strip(" ").strip("\n").replace("  ", " "))
f.close()
print(len(test_med_query_f_metamap_list), len(val_med_query_f_metamap_list))

246 569


In [7]:
test_queries = {'med':[], 'nonmed': []}
val_queries = {'med':[], 'nonmed': []}
med_total, nonmed_total = 0, 0
for key, value in test[0].items():
    vid = next(iter(value))
    ismed = False
    newkey = key.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(">", "").replace(",", "").replace(".", "").replace("/", "")
    if newkey  in test_med_query_f_metamap_list:
        ismed = True
    if ismed:
        med_total += 1
        test_queries['med'].append([key, vid.replace(".mp4", ""), value[vid]['bounds']])
    else:
        nonmed_total += 1
        test_queries['nonmed'].append([key, vid.replace(".mp4", ""), value[vid]['bounds']])
    
print(med_total, nonmed_total)
med_total, nonmed_total = 0, 0
for key, value in val[0].items():
    vid = next(iter(value))
    ismed = False
    newkey = key.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(">", "").replace(",", "").replace(".", "").replace("/", "")
    if newkey  in val_med_query_f_metamap_list:
        ismed = True
    if ismed:
        med_total += 1
        val_queries['med'].append([key, vid.replace(".mp4", ""), value[vid]['bounds']])
    else:
        nonmed_total += 1
        val_queries['nonmed'].append([key, vid.replace(".mp4", ""), value[vid]['bounds']])

print(med_total, nonmed_total)

246 7372
569 14675


In [8]:
vid_duration_json = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/video_duration.json"
video_duration_dict = load_jsonl(vid_duration_json)[0]

In [9]:
clip_pred_med = load_jsonl(f'/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/five_labeled_pred_med_from_gt_vid_dict.json')

In [10]:
#select_option = "med_queries"
select_option = "med_samples_clip_from_gt"
# select_option = "med_samples_clip_full_rand"
#select_option = "tv shows"

In [11]:
for folder in folders:
    print(f"{folder}\n")

    test_result_json_path = f"{folder}/all_data_test_moment_retrieval_BEST_with_audio.json"
    val_result_json_path = f"{folder}/all_data_val_moment_retrieval_BEST_with_audio.json"
    # test_audio_relevance_json_path = f"{folder}/test_audio_relevance_score.json"
    # val_audio_relevance_json_path = f"{folder}/val_audio_relevance_score.json"
    test_audio_relevance_json_path = f"{folder}/test_audio_relevance_score_paragraph_semantic_search.json"
    val_audio_relevance_json_path = f"{folder}/val_audio_relevance_score_paragraph_semantic_search.json"

    test_result = load_result_json(test_result_json_path)
    val_result = load_result_json(val_result_json_path)
    test_audio_relevance_score = load_result_json(test_audio_relevance_json_path)
    val_audio_relevance_score = load_result_json(val_audio_relevance_json_path)

    for value in test_audio_relevance_score:
        if 'score' not in value:
            continue
        if value['score'] > 3:
            value['score'] = 3
        elif value['score'] < 1:
            value['score'] = 1
        else:
            value['score'] = 2
    for value in val_audio_relevance_score:
        if 'score' not in value:
            continue
        if value['score'] > 3:
            value['score'] = 3
        elif value['score'] < 1:
            value['score'] = 1
        else:
            value['score'] = 2


    med_iou_thresh_3, med_iou_thresh_5, med_iou_thresh_7, med_cnt = 0, 0, 0, 0
    nonmed_iou_thresh_3, nonmed_iou_thresh_5, nonmed_iou_thresh_7, nonmed_cnt, cnt = 0, 0, 0, 0, 0
    score = {'med':{'low' : [0, 0, 0, 0], 'high' : [0, 0, 0, 0], "unknown" : [0, 0,0, 0]}, 'nonmed':{'low' : [0, 0, 0, 0], 'high' : [0, 0, 0, 0], "unknown" : [0, 0,0, 0]}}
    idx = 0
    examples = ""
    for key, value in val[0].items():
        if key in val_result:
            vid = next(iter(value))
            iou = compute_iou(value[vid]['bounds'], val_result[key][vid]['bounds'])
            cnt += 1
            ismed = False
            if select_option == "tv shows":
                if "house" in vid or "grey" in vid:
                    ismed = True
            elif select_option == "med_samples_clip_full_rand":
                if clip_pred_med[0][vid] == "med":
                    ismed = True
            elif select_option == "med_samples_clip_from_gt":
                for qc in clip_pred_med[0][vid.replace(".mp4", "")]:
                    #print(qc)
                    if key in qc and qc[key] == "med":
                        ismed = True
            elif select_option == "med_queries":
                newkey = key.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(">", "").replace(",", "").replace(".", "").replace("/", "")
                if newkey  in val_med_query_f_metamap_list:
                    ismed = True
            category = "unknown"
            for element in val_audio_relevance_score:
                if element['question'] == key and element['video']== vid:
                    if 'score' not in element:
                        category = "unknown"
                        break
                    if element['score'] == 1:
                        category = "low"
                    elif element['score'] == 3:
                        category = "high"
                    if category == "high" and idx < 5:
                        examples += f"{vid} {key} {iou} {value[vid]['bounds']} {val_result[key][vid]['bounds']}\n{element['sub']}\n"
                        idx += 1
                    break
            
            
            if ismed:
                med_cnt += 1
                #sum
                score['med'][category][3] += 1
                if iou >= 0.3:
                    med_iou_thresh_3 += 1 
                    score['med'][category][0] += 1
                if iou >= 0.5:
                    med_iou_thresh_5 += 1 
                    score['med'][category][1] += 1
                if iou >= 0.7:
                    med_iou_thresh_7 += 1
                    score['med'][category][2] += 1
            else:
                nonmed_cnt += 1
                score['nonmed'][category][3] += 1
                if iou >= 0.3:
                    nonmed_iou_thresh_3 += 1 
                    score['nonmed'][category][0] += 1
                if iou >= 0.5:
                    nonmed_iou_thresh_5 += 1 
                    score['nonmed'][category][1] += 1
                if iou >= 0.7:
                    nonmed_iou_thresh_7 += 1
                    score['nonmed'][category][2] += 1
    print(f"\nMed VAL IoU=0.3: {med_iou_thresh_3/(med_cnt) * 100}\nIoU=0.5: {med_iou_thresh_5/(med_cnt) * 100}\nIoU=0.7: {med_iou_thresh_7/(med_cnt) * 100}\ntotal queries:{med_cnt}/ {cnt}")
    print(f"\nNonmed VAL IoU=0.3: {nonmed_iou_thresh_3/(nonmed_cnt) * 100}\nIoU=0.5: {nonmed_iou_thresh_5/(nonmed_cnt) * 100}\nIoU=0.7: {nonmed_iou_thresh_7/(nonmed_cnt) * 100}\ntotal queries:{nonmed_cnt} / {cnt}\n")
    for category in ['low', 'high']:
            iou3 = (score['med'][category][0] + score['nonmed'][category][0]) / (score['med'][category][3] + score['nonmed'][category][3]) * 100
            iou5 = (score['med'][category][1] + score['nonmed'][category][1]) / (score['med'][category][3] + score['nonmed'][category][3]) * 100
            iou7 = (score['med'][category][2] + score['nonmed'][category][2]) / (score['med'][category][3] + score['nonmed'][category][3]) * 100
            print(f"All VAL {category} IoU=0.3: {iou3}, IoU=0.5: {iou5}, IoU=0.7: {iou7}")
    print(f"\nMed VAL low relevance to audio, IoU=0.3: {score['med']['low'][0]/ score['med']['low'][3] * 100} IoU=0.5: {score['med']['low'][1]/ score['med']['low'][3] * 100} IoU=0.7: {score['med']['low'][2]/ score['med']['low'][3] * 100}")
    print(f"Med VAL high relevance to audio, IoU=0.3: {score['med']['high'][0]/ score['med']['high'][3] * 100} IoU=0.5: {score['med']['high'][1]/ score['med']['high'][3] * 100} IoU=0.7: {score['med']['high'][2]/ score['med']['high'][3] * 100}")

    print(f"\nNonMed VAL low relevance to audio, IoU=0.3: {score['nonmed']['low'][0]/ score['nonmed']['low'][3] * 100} IoU=0.5: {score['nonmed']['low'][1]/ score['nonmed']['low'][3] * 100} IoU=0.7: {score['nonmed']['low'][2]/ score['nonmed']['low'][3] * 100}")
    print(f"NonMed VAL high relevance to audio, IoU=0.3: {score['nonmed']['high'][0]/ score['nonmed']['high'][3] * 100} IoU=0.5: {score['nonmed']['high'][1]/ score['nonmed']['high'][3] * 100} IoU=0.7: {score['nonmed']['high'][2]/ score['nonmed']['high'][3] * 100}")
    

    med_iou_thresh_3, med_iou_thresh_5, med_iou_thresh_7, med_cnt = 0, 0, 0, 0
    nonmed_iou_thresh_3, nonmed_iou_thresh_5, nonmed_iou_thresh_7, nonmed_cnt, cnt = 0, 0, 0, 0, 0
    score = {'med':{'low' : [0, 0, 0, 0], 'mid' : [0, 0, 0, 0], 'high' : [0, 0, 0, 0], "unknown" : [0, 0,0, 0]}, 'nonmed':{'low' : [0, 0, 0, 0], 'mid' : [0, 0, 0, 0], 'high' : [0, 0, 0, 0], "unknown" : [0, 0,0, 0]}}
    examples = ""
    idx = 0
    for key, value in test[0].items():
        if key in test_result:
            #print(key)
            vid = next(iter(value))
            iou = compute_iou(value[vid]['bounds'], test_result[key][vid]['bounds'])
            cnt += 1
            ismed = False
            if select_option == "tv shows":
                if "house" in vid or "grey" in vid:
                    ismed = True
            elif select_option == "med_samples_clip_full_rand":
                if clip_pred_med[0][vid] == "med":
                    ismed = True
            elif select_option == "med_samples_clip_from_gt":
                for qc in clip_pred_med[0][vid.replace(".mp4", "")]:
                    #print(qc)
                    if key in qc and qc[key] == "med":
                        ismed = True
            elif select_option == "med_queries":
                newkey = key.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(">", "").replace(",", "").replace(".", "").replace("/", "")
                if newkey  in test_med_query_f_metamap_list:
                    ismed = True
            category = "unknown"
            for element in test_audio_relevance_score:
                if element['question'] == key and element['video']== vid:
                    if 'score' not in element:
                        category = "unknown"
                        break
                    if element['score'] == 1:
                        category = "low"
                    elif element['score'] == 2:
                        category = "mid"
                    elif element['score'] == 3:
                        category = "high"
                    if category == "high" and idx < 5:
                        examples += f"{vid} {key} {iou} {value[vid]['bounds']} {test_result[key][vid]['bounds']}\n{element['sub']}\n"
                        idx += 1
                    break
            
            
            if ismed:
                med_cnt += 1
                #sum
                score['med'][category][3] += 1
                if iou >= 0.3:
                    med_iou_thresh_3 += 1 
                    score['med'][category][0] += 1
                if iou >= 0.5:
                    med_iou_thresh_5 += 1 
                    score['med'][category][1] += 1
                if iou >= 0.7:
                    med_iou_thresh_7 += 1
                    score['med'][category][2] += 1
            else:
                nonmed_cnt += 1
                score['nonmed'][category][3] += 1
                if iou >= 0.3:
                    nonmed_iou_thresh_3 += 1 
                    score['nonmed'][category][0] += 1
                if iou >= 0.5:
                    nonmed_iou_thresh_5 += 1 
                    score['nonmed'][category][1] += 1
                if iou >= 0.7:
                    nonmed_iou_thresh_7 += 1
                    score['nonmed'][category][2] += 1
    print(f"Med TEST IoU=0.3: {med_iou_thresh_3/(med_cnt) * 100}\nIoU=0.5: {med_iou_thresh_5/(med_cnt) * 100}\nIoU=0.7: {med_iou_thresh_7/(med_cnt) * 100}\ntotal queries:{med_cnt}/ {cnt}")
    print(f"\nNonmed TEST IoU=0.3:\n {nonmed_iou_thresh_3/(nonmed_cnt) * 100}\nIoU=0.5: {nonmed_iou_thresh_5/(nonmed_cnt) * 100}\nIoU=0.7: {nonmed_iou_thresh_7/(nonmed_cnt) * 100}\ntotal queries:{nonmed_cnt} / {cnt}\n")
    
    for category in ['low', 'mid', 'high']:
        iou3 = (score['med'][category][0] + score['nonmed'][category][0]) / (score['med'][category][3] + score['nonmed'][category][3]) * 100
        iou5 = (score['med'][category][1] + score['nonmed'][category][1]) / (score['med'][category][3] + score['nonmed'][category][3]) * 100
        iou7 = (score['med'][category][2] + score['nonmed'][category][2]) / (score['med'][category][3] + score['nonmed'][category][3]) * 100
        print(f"All TEST {category} IoU=0.3: {iou3}, IoU=0.5: {iou5}, IoU=0.7: {iou7}")

    print(f"\nMed TEST low relevance to audio, IoU=0.3: {score['med']['low'][0]/ score['med']['low'][3] * 100} IoU=0.5: {score['med']['low'][1]/ score['med']['low'][3] * 100} IoU=0.7: {score['med']['low'][2]/ score['med']['low'][3] * 100}")
    print(f"Med TEST mid relevance to audio, IoU=0.3: {score['med']['mid'][0]/ score['med']['mid'][3] * 100} IoU=0.5: {score['med']['mid'][1]/ score['med']['mid'][3] * 100} IoU=0.7: {score['med']['mid'][2]/ score['med']['mid'][3] * 100}")
    print(f"Med TEST high relevance to audio, IoU=0.3: {score['med']['high'][0]/ score['med']['high'][3] * 100} IoU=0.5: {score['med']['high'][1]/ score['med']['high'][3] * 100} IoU=0.7: {score['med']['high'][2]/ score['med']['high'][3] * 100}")

    print(f"\nNonMed TEST low relevance to audio, IoU=0.3: {score['nonmed']['low'][0]/ score['nonmed']['low'][3] * 100} IoU=0.5: {score['nonmed']['low'][1]/ score['nonmed']['low'][3] * 100} IoU=0.7: {score['nonmed']['low'][2]/ score['nonmed']['low'][3] * 100}")
    print(f"NonMed TEST mid relevance to audio, IoU=0.3: {score['nonmed']['mid'][0]/ score['nonmed']['mid'][3] * 100} IoU=0.5: {score['nonmed']['mid'][1]/ score['nonmed']['mid'][3] * 100} IoU=0.7: {score['nonmed']['mid'][2]/ score['nonmed']['mid'][3] * 100}")
    print(f"NonMed TEST high relevance to audio, IoU=0.3: {score['nonmed']['high'][0]/ score['nonmed']['high'][3] * 100} IoU=0.5: {score['nonmed']['high'][1]/ score['nonmed']['high'][3] * 100} IoU=0.7: {score['nonmed']['high'][2]/ score['nonmed']['high'][3] * 100}")
    
    print(f"{examples}")
    

    print("\n\n")

/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_gpt_query_type_experiment/baseline


Med VAL IoU=0.3: 4.2561065877128055
IoU=0.5: 1.8134715025906734
IoU=0.7: 0.9252405625462621
total queries:2702/ 15244

Nonmed VAL IoU=0.3: 10.620315739116569
IoU=0.5: 5.031095519055972
IoU=0.7: 2.798596715037474
total queries:12542 / 15244

All VAL low IoU=0.3: 8.265027322404372, IoU=0.5: 3.859289617486339, IoU=0.7: 2.3224043715846996
All VAL high IoU=0.3: 11.059441973311767, IoU=0.5: 4.771532551556813, IoU=0.7: 2.3857662757784066

Med VAL low relevance to audio, IoU=0.3: 2.923976608187134 IoU=0.5: 1.364522417153996 IoU=0.7: 0.9746588693957114
Med VAL high relevance to audio, IoU=0.3: 4.6893317702227435 IoU=0.5: 2.3446658851113718 IoU=0.7: 0.9378663540445485

NonMed VAL low relevance to audio, IoU=0.3: 9.399585921325052 IoU=0.5: 4.3892339544513455 IoU=0.7: 2.608695652173913
NonMed VAL high relevance to audio, IoU=0.3: 12.38700219887613 IoU=0.5: 5.277302711947227 IoU=0.7: 2.687515269973124

In [12]:
folders = ['/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_gpt_query_type_textsim_experiment/baseline', 
           '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_gpt_query_type_textsim_experiment/audio_high', 
           '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_gpt_query_type_textsim_experiment/audio_low', 
           '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_gpt_query_type_textsim_experiment/sample_audio_high',
           '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_gpt_query_type_textsim_experiment/scratch_baseline',
           '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_gpt_query_type_textsim_experiment/scratch_baseline_kl',
           '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_gpt_query_type_textsim_experiment/scratch_baseline_kl_div2',
           '/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_gpt_query_type_textsim_experiment/scratch_baseline_mse']

In [13]:
for folder in folders:
    print(f"{folder}\n")

    test_result_json_path = f"{folder}/all_data_test_moment_retrieval_BEST_with_audio.json"
    val_result_json_path = f"{folder}/all_data_val_moment_retrieval_BEST_with_audio.json"
    # test_audio_relevance_json_path = f"{folder}/test_audio_relevance_score.json"
    # val_audio_relevance_json_path = f"{folder}/val_audio_relevance_score.json"
    test_audio_relevance_json_path = f"{folder}/test_audio_relevance_score_paragraph_semantic_search.json"
    val_audio_relevance_json_path = f"{folder}/val_audio_relevance_score_paragraph_semantic_search.json"

    test_result = load_result_json(test_result_json_path)
    val_result = load_result_json(val_result_json_path)
    test_audio_relevance_score = load_result_json(test_audio_relevance_json_path)
    val_audio_relevance_score = load_result_json(val_audio_relevance_json_path)

    for value in test_audio_relevance_score:
        if 'score' not in value:
            continue
        if value['score'] > 3:
            value['score'] = 3
        elif value['score'] < 1:
            value['score'] = 1
        else:
            value['score'] = 2
    for value in val_audio_relevance_score:
        if 'score' not in value:
            continue
        if value['score'] > 3:
            value['score'] = 3
        elif value['score'] < 1:
            value['score'] = 1
        else:
            value['score'] = 2

    _iou_thresh_3, _iou_thresh_5, _iou_thresh_7, _cnt = 0, 0, 0, 0
    med_iou_thresh_3, med_iou_thresh_5, med_iou_thresh_7, med_cnt = 0, 0, 0, 0
    nonmed_iou_thresh_3, nonmed_iou_thresh_5, nonmed_iou_thresh_7, nonmed_cnt, cnt = 0, 0, 0, 0, 0
    score = {'med':{'low' : [0, 0, 0, 0], 'high' : [0, 0, 0, 0], "unknown" : [0, 0,0, 0]}, 'nonmed':{'low' : [0, 0, 0, 0], 'high' : [0, 0, 0, 0], "unknown" : [0, 0,0, 0]}}
    idx = 0
    examples = ""
    for key, value in val[0].items():
        if key in val_result:
            vid = next(iter(value))
            iou = compute_iou(value[vid]['bounds'], val_result[key][vid]['bounds'])
            cnt += 1
            ismed = False
            if select_option == "tv shows":
                if "house" in vid or "grey" in vid:
                    ismed = True
            elif select_option == "med_samples_clip_full_rand":
                if clip_pred_med[0][vid] == "med":
                    ismed = True
            elif select_option == "med_samples_clip_from_gt":
                for qc in clip_pred_med[0][vid.replace(".mp4", "")]:
                    #print(qc)
                    if key in qc and qc[key] == "med":
                        ismed = True
            elif select_option == "med_queries":
                newkey = key.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(">", "").replace(",", "").replace(".", "").replace("/", "")
                if newkey  in val_med_query_f_metamap_list:
                    ismed = True
            category = "unknown"
            for element in val_audio_relevance_score:
                if element['question'] == key and element['video']== vid:
                    if 'score' not in element:
                        category = "unknown"
                        break
                    if element['score'] == 1:
                        category = "low"
                    elif element['score'] == 3:
                        category = "high"
                    if category == "high" and idx < 5:
                        examples += f"{vid} {key} {iou} {value[vid]['bounds']} {val_result[key][vid]['bounds']}\n{element['sub']}\n"
                        idx += 1
                    break
            _cnt += 1
            if iou >= 0.3:
                _iou_thresh_3 += 1 
            if iou >= 0.5:
                _iou_thresh_5 += 1 
            if iou >= 0.7:
                _iou_thresh_7 += 1

            if ismed:
                med_cnt += 1
                #sum
                score['med'][category][3] += 1
                if iou >= 0.3:
                    med_iou_thresh_3 += 1 
                    score['med'][category][0] += 1
                if iou >= 0.5:
                    med_iou_thresh_5 += 1 
                    score['med'][category][1] += 1
                if iou >= 0.7:
                    med_iou_thresh_7 += 1
                    score['med'][category][2] += 1
            else:
                nonmed_cnt += 1
                score['nonmed'][category][3] += 1
                if iou >= 0.3:
                    nonmed_iou_thresh_3 += 1 
                    score['nonmed'][category][0] += 1
                if iou >= 0.5:
                    nonmed_iou_thresh_5 += 1 
                    score['nonmed'][category][1] += 1
                if iou >= 0.7:
                    nonmed_iou_thresh_7 += 1
                    score['nonmed'][category][2] += 1
    print(f"\All VAL IoU=0.3: {_iou_thresh_3/(_cnt) * 100}\nIoU=0.5: {_iou_thresh_5/(_cnt) * 100}\nIoU=0.7: {_iou_thresh_7/(_cnt) * 100}\ntotal queries:{_cnt}/ {cnt}")

    print(f"\nMed VAL IoU=0.3: {med_iou_thresh_3/(med_cnt) * 100}\nIoU=0.5: {med_iou_thresh_5/(med_cnt) * 100}\nIoU=0.7: {med_iou_thresh_7/(med_cnt) * 100}\ntotal queries:{med_cnt}/ {cnt}")
    print(f"\nNonmed VAL IoU=0.3: {nonmed_iou_thresh_3/(nonmed_cnt) * 100}\nIoU=0.5: {nonmed_iou_thresh_5/(nonmed_cnt) * 100}\nIoU=0.7: {nonmed_iou_thresh_7/(nonmed_cnt) * 100}\ntotal queries:{nonmed_cnt} / {cnt}\n")
    for category in ['low', 'high']:
            iou3 = (score['med'][category][0] + score['nonmed'][category][0]) / (score['med'][category][3] + score['nonmed'][category][3]) * 100
            iou5 = (score['med'][category][1] + score['nonmed'][category][1]) / (score['med'][category][3] + score['nonmed'][category][3]) * 100
            iou7 = (score['med'][category][2] + score['nonmed'][category][2]) / (score['med'][category][3] + score['nonmed'][category][3]) * 100
            print(f"All VAL {category} IoU=0.3: {iou3}, IoU=0.5: {iou5}, IoU=0.7: {iou7}")
    
    print(f"\nMed VAL low relevance to audio, IoU=0.3: {score['med']['low'][0]/ score['med']['low'][3] * 100} IoU=0.5: {score['med']['low'][1]/ score['med']['low'][3] * 100} IoU=0.7: {score['med']['low'][2]/ score['med']['low'][3] * 100}")
    print(f"Med VAL high relevance to audio, IoU=0.3: {score['med']['high'][0]/ score['med']['high'][3] * 100} IoU=0.5: {score['med']['high'][1]/ score['med']['high'][3] * 100} IoU=0.7: {score['med']['high'][2]/ score['med']['high'][3] * 100}")

    print(f"\nNonMed VAL low relevance to audio, IoU=0.3: {score['nonmed']['low'][0]/ score['nonmed']['low'][3] * 100} IoU=0.5: {score['nonmed']['low'][1]/ score['nonmed']['low'][3] * 100} IoU=0.7: {score['nonmed']['low'][2]/ score['nonmed']['low'][3] * 100}")
    print(f"NonMed VAL high relevance to audio, IoU=0.3: {score['nonmed']['high'][0]/ score['nonmed']['high'][3] * 100} IoU=0.5: {score['nonmed']['high'][1]/ score['nonmed']['high'][3] * 100} IoU=0.7: {score['nonmed']['high'][2]/ score['nonmed']['high'][3] * 100}")
    
    _iou_thresh_3, _iou_thresh_5, _iou_thresh_7, _cnt = 0, 0, 0, 0
    med_iou_thresh_3, med_iou_thresh_5, med_iou_thresh_7, med_cnt = 0, 0, 0, 0
    nonmed_iou_thresh_3, nonmed_iou_thresh_5, nonmed_iou_thresh_7, nonmed_cnt, cnt = 0, 0, 0, 0, 0
    score = {'med':{'low' : [0, 0, 0, 0], 'mid' : [0, 0, 0, 0], 'high' : [0, 0, 0, 0], "unknown" : [0, 0,0, 0]}, 'nonmed':{'low' : [0, 0, 0, 0], 'mid' : [0, 0, 0, 0], 'high' : [0, 0, 0, 0], "unknown" : [0, 0,0, 0]}}
    examples = ""
    idx = 0
    for key, value in test[0].items():
        if key in test_result:
            #print(key)
            vid = next(iter(value))
            iou = compute_iou(value[vid]['bounds'], test_result[key][vid]['bounds'])
            cnt += 1
            ismed = False
            if select_option == "tv shows":
                if "house" in vid or "grey" in vid:
                    ismed = True
            elif select_option == "med_samples_clip_full_rand":
                if clip_pred_med[0][vid] == "med":
                    ismed = True
            elif select_option == "med_samples_clip_from_gt":
                for qc in clip_pred_med[0][vid.replace(".mp4", "")]:
                    #print(qc)
                    if key in qc and qc[key] == "med":
                        ismed = True
            elif select_option == "med_queries":
                newkey = key.lower().replace(" ", "").replace("?", "").replace("\n", "").replace("-", "").replace("'", "").replace("\"", "").replace(">", "").replace(",", "").replace(".", "").replace("/", "")
                if newkey  in test_med_query_f_metamap_list:
                    ismed = True
            category = "unknown"
            for element in test_audio_relevance_score:
                if element['question'] == key and element['video']== vid:
                    if 'score' not in element:
                        category = "unknown"
                        break
                    if element['score'] == 1:
                        category = "low"
                    elif element['score'] == 2:
                        category = "mid"
                    elif element['score'] == 3:
                        category = "high"
                    if category == "high" and idx < 5:
                        examples += f"{vid} {key} {iou} {value[vid]['bounds']} {test_result[key][vid]['bounds']}\n{element['sub']}\n"
                        idx += 1
                    break
            _cnt += 1
            if iou >= 0.3:
                _iou_thresh_3 += 1 
            if iou >= 0.5:
                _iou_thresh_5 += 1 
            if iou >= 0.7:
                _iou_thresh_7 += 1
            
            if ismed:
                med_cnt += 1
                #sum
                score['med'][category][3] += 1
                if iou >= 0.3:
                    med_iou_thresh_3 += 1 
                    score['med'][category][0] += 1
                if iou >= 0.5:
                    med_iou_thresh_5 += 1 
                    score['med'][category][1] += 1
                if iou >= 0.7:
                    med_iou_thresh_7 += 1
                    score['med'][category][2] += 1
            else:
                nonmed_cnt += 1
                score['nonmed'][category][3] += 1
                if iou >= 0.3:
                    nonmed_iou_thresh_3 += 1 
                    score['nonmed'][category][0] += 1
                if iou >= 0.5:
                    nonmed_iou_thresh_5 += 1 
                    score['nonmed'][category][1] += 1
                if iou >= 0.7:
                    nonmed_iou_thresh_7 += 1
                    score['nonmed'][category][2] += 1
    print(f"\All TEST IoU=0.3: {_iou_thresh_3/(_cnt) * 100}\nIoU=0.5: {_iou_thresh_5/(_cnt) * 100}\nIoU=0.7: {_iou_thresh_7/(_cnt) * 100}\ntotal queries:{_cnt}/ {cnt}")

    print(f"Med TEST IoU=0.3: {med_iou_thresh_3/(med_cnt) * 100}\nIoU=0.5: {med_iou_thresh_5/(med_cnt) * 100}\nIoU=0.7: {med_iou_thresh_7/(med_cnt) * 100}\ntotal queries:{med_cnt}/ {cnt}")
    print(f"\nNonmed TEST IoU=0.3:\n {nonmed_iou_thresh_3/(nonmed_cnt) * 100}\nIoU=0.5: {nonmed_iou_thresh_5/(nonmed_cnt) * 100}\nIoU=0.7: {nonmed_iou_thresh_7/(nonmed_cnt) * 100}\ntotal queries:{nonmed_cnt} / {cnt}\n")
    
    for category in ['low', 'mid', 'high']:
        iou3 = (score['med'][category][0] + score['nonmed'][category][0]) / (score['med'][category][3] + score['nonmed'][category][3]) * 100
        iou5 = (score['med'][category][1] + score['nonmed'][category][1]) / (score['med'][category][3] + score['nonmed'][category][3]) * 100
        iou7 = (score['med'][category][2] + score['nonmed'][category][2]) / (score['med'][category][3] + score['nonmed'][category][3]) * 100
        print(f"All TEST {category} IoU=0.3: {iou3}, IoU=0.5: {iou5}, IoU=0.7: {iou7}")

    print(f"\nMed TEST low relevance to audio, IoU=0.3: {score['med']['low'][0]/ score['med']['low'][3] * 100} IoU=0.5: {score['med']['low'][1]/ score['med']['low'][3] * 100} IoU=0.7: {score['med']['low'][2]/ score['med']['low'][3] * 100}")
    print(f"Med TEST mid relevance to audio, IoU=0.3: {score['med']['mid'][0]/ score['med']['mid'][3] * 100} IoU=0.5: {score['med']['mid'][1]/ score['med']['mid'][3] * 100} IoU=0.7: {score['med']['mid'][2]/ score['med']['mid'][3] * 100}")
    print(f"Med TEST high relevance to audio, IoU=0.3: {score['med']['high'][0]/ score['med']['high'][3] * 100} IoU=0.5: {score['med']['high'][1]/ score['med']['high'][3] * 100} IoU=0.7: {score['med']['high'][2]/ score['med']['high'][3] * 100}")

    print(f"\nNonMed TEST low relevance to audio, IoU=0.3: {score['nonmed']['low'][0]/ score['nonmed']['low'][3] * 100} IoU=0.5: {score['nonmed']['low'][1]/ score['nonmed']['low'][3] * 100} IoU=0.7: {score['nonmed']['low'][2]/ score['nonmed']['low'][3] * 100}")
    print(f"NonMed TEST mid relevance to audio, IoU=0.3: {score['nonmed']['mid'][0]/ score['nonmed']['mid'][3] * 100} IoU=0.5: {score['nonmed']['mid'][1]/ score['nonmed']['mid'][3] * 100} IoU=0.7: {score['nonmed']['mid'][2]/ score['nonmed']['mid'][3] * 100}")
    print(f"NonMed TEST high relevance to audio, IoU=0.3: {score['nonmed']['high'][0]/ score['nonmed']['high'][3] * 100} IoU=0.5: {score['nonmed']['high'][1]/ score['nonmed']['high'][3] * 100} IoU=0.7: {score['nonmed']['high'][2]/ score['nonmed']['high'][3] * 100}")
    
    print(f"{examples}")
    

    print("\n\n")

/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_gpt_query_type_textsim_experiment/baseline



\All VAL IoU=0.3: 9.492259249540803
IoU=0.5: 4.460771451062713
IoU=0.7: 2.4665442141170297
total queries:15244/ 15244

Med VAL IoU=0.3: 4.2561065877128055
IoU=0.5: 1.8134715025906734
IoU=0.7: 0.9252405625462621
total queries:2702/ 15244

Nonmed VAL IoU=0.3: 10.620315739116569
IoU=0.5: 5.031095519055972
IoU=0.7: 2.798596715037474
total queries:12542 / 15244

All VAL low IoU=0.3: 8.265027322404372, IoU=0.5: 3.859289617486339, IoU=0.7: 2.3224043715846996
All VAL high IoU=0.3: 11.059441973311767, IoU=0.5: 4.771532551556813, IoU=0.7: 2.3857662757784066

Med VAL low relevance to audio, IoU=0.3: 2.923976608187134 IoU=0.5: 1.364522417153996 IoU=0.7: 0.9746588693957114
Med VAL high relevance to audio, IoU=0.3: 4.6893317702227435 IoU=0.5: 2.3446658851113718 IoU=0.7: 0.9378663540445485

NonMed VAL low relevance to audio, IoU=0.3: 9.399585921325052 IoU=0.5: 4.3892339544513455 IoU=0.7: 2.608695652173913
NonMed VAL high relevance to audio, IoU=0.3: 12.38700219887613 IoU=0.5: 5.277302711947227 IoU=0.